# 학습 관련 기술들

이번 장에서 알아볼 내용
- 최적화 방법
- 가중치 매개변수 초깃값 (전에는 random으로 설정)
- 하이퍼파라미터 설정 방법 (node, lr, batch size....)
- 오버피팅 방지 (가중치 감소, 드랍아웃 등 정규화 방법)
- 배치 정규화

In [1]:
# 라이브러리
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
path = '/content/drive/MyDrive/deep-learning-from-scratch-master'
os.chdir(path)
sys.path.append(os.chdir)

# 매개변수 갱신

매개변수의 최적값을 찾는 이런 과정을 최적화(Optimization)라고 한다.

최적의 매개변수를 찾을 단서로 매개변수의 기울기(1.수치미분 2.오차역전파법)을 앞서 살펴봤음

GD의 수식

$W \leftarrow W - \eta\frac{\partial{L}}{\partial{W}}$

아래와 같이 구현하면 SGD의 인스턴스에 매개변수와 기울기 정보를 전달하면 매개변수가 update 

참고내용 Lasagne 프레임워크의 최적화 기법들 

https://github.com/Lasagne/Lasagne/blob/master/lasagne/updates.py

In [3]:
# Gradient Descent 구현

class SGD(object):
    def __init__(self, lr = 0.01):
        self.lr = lr  # 인스턴스 변수로 learning rate 

    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

SGD의 문제점